In [3]:
A=((1),[2,3],[4])
A[1][1]

3

In [13]:
L = ['c', 'd']
L = L.append(['a','b'])
print(L)

None


In [16]:
A=["hard rock",10,1.2]
del(A[1])
A

['hard rock', 1.2]

In [17]:
 len(("disco",10))

2

In [20]:
V={'A','B'}
V.add('C')
V

{'A', 'B', 'C'}

In [21]:
'A' in {'A','B'}

True

In [22]:
class Points(object):
    def __init__(self, x, y):
        self.x = x 
        self.y = y 


    def print_point(self): 
        print('x=', self.x, ' y=', self.y) 


p1 = Points("A", "B") 
p1.print_point()


x= A  y= B


In [23]:
for i, x in enumerate(['A', 'B', 'C']): 
    print(i + 1, x)

1 A
2 B
3 C


In [24]:
class Points(object): 
    def __init__(self, x, y): 
        self.x = x 
        self.y = y 
    def print_point(self): 
        print('x=', self.x, ' y=', self.y) 


p2 = Points(1, 2) 
p2.x = 2 
p2.print_point()

x= 2  y= 2


In [25]:
import numpy as np

a=np.array([-1,1]) 

b=np.array([1,1]) 

np.dot(a,b) 

0

In [27]:
X=np.array([[1,0,1],[2,2,2]]) 

out=X[0:2,2]
out

array([1, 2])

In [29]:
X=np.array([[1,0],[0,1]])

Y=np.array([[2,2],[2,2]]) 

out=np.dot(X,Y) 
out

array([[2, 2],
       [2, 2]])

In [31]:
1//2

0

In [32]:
'1,2,3,4'.split(',')

['1', '2', '3', '4']

In [33]:
x = 7
if(x!=1): 

 print('Hi') 

else: 

 print('Hello')

print('Mike')

Hi
Mike


In [40]:
class Rectangle(object): 

    def __init__(self,width=2,height =3,color='r'):

        self.height=height 

        self.width=width 

        self.color=color 

    def drawRectangle(self): 

        import matplotlib.pyplot as plt 

        plt.gca().add_patch(plt.Rectangle((0, 0),self.width, self.height ,fc=self.color)) 

        plt.axis('scaled')
        plt.show()
        
Rectangle.drawRectangle(2,3)

TypeError: Rectangle.drawRectangle() takes 1 positional argument but 2 were given

In [41]:
a=np.array([0,1,0,1,0]) 

b=np.array([1,0,1,0,1]) 

a*b

array([0, 0, 0, 0, 0])

In [42]:
a=np.array([1,1,1,1,1]) 

a+10

array([11, 11, 11, 11, 11])

In [141]:

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

# Code for ETL operations on Country-GDP data
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
exchange_rate_path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'
exchange_rate_csv = './exchange_rate.csv'
table_attribs = ["Bank name", "Market cap"]
final_table_attribs = ["Name", "MC_USD_Billion","MC_GBP_Billion","MC_EUR_Billion","MC_INR_Billion"]
db_name = 'Banks.db'
table_name = 'Largest_banks'
output_path = './Largest_banks_data.csv'
log_file = 'code_log.txt'

def log_progress(message):
    ''' This function logs the mentioned message at a given stage of the 
    code execution to a log file. Function returns nothing.'''
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

def extract(url, table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
#             print(col[1].find_all('a')[1].text)
            if col[0] is not None:
                data_dict = {"Bank name": col[1].find_all('a')[1].text,
                             "Market cap": float(col[2].contents[0])}
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df1,df], ignore_index=True)
    return df

def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    
    # Load exchange rate information from CSV file
    exchange_rates = pd.read_csv(csv_path)
    
    # Convert exchange rates to a dictionary for easy access
    rates = dict(zip(exchange_rates['Currency'], exchange_rates['Rate']))
    
    # Add new columns for Market Cap in GBP, EUR, and INR
    df['MC_GBP_Billion'] = (df['Market cap'].astype(float) * rates['GBP']).round(2)
    df['MC_EUR_Billion'] = (df['Market cap'].astype(float) * rates['EUR']).round(2)
    df['MC_INR_Billion'] = (df['Market cap'].astype(float) * rates['INR']).round(2)
    
    return df

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path, index=False)

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final dataframe to as a database table
    with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, sql_connection):
    ''' This function runs the stated query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)
        
log_progress('Preliminaries complete. Initiating ETL process')
df = extract(url, table_attribs)
log_progress('Data extraction complete. Initiating Transformation process')
df = transform(df,exchange_rate_csv)
log_progress('Data transformation complete. Initiating loading process')
load_to_csv(df, output_path)
log_progress('Data saved to CSV file')
sql_connection = sqlite3.connect('Banks.db')
log_progress('SQL Connection initiated.')
load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database as table. Running the query')
query_statement = f"SELECT * from {table_name}"
run_query(query_statement, sql_connection)
log_progress('Process Complete.')
sql_connection.close()


JPMorgan Chase
Bank of America
Industrial and Commercial Bank of China
Agricultural Bank of China
HDFC Bank
Wells Fargo
HSBC Holdings PLC
Morgan Stanley
China Construction Bank
Bank of China
SELECT * from Largest_banks
                                 Bank name  Market cap  MC_GBP_Billion  \
0                            Bank of China      136.81          109.45   
1                  China Construction Bank      139.82          111.86   
2                           Morgan Stanley      140.83          112.66   
3                        HSBC Holdings PLC      148.90          119.12   
4                              Wells Fargo      155.87          124.70   
5                                HDFC Bank      157.91          126.33   
6               Agricultural Bank of China      160.68          128.54   
7  Industrial and Commercial Bank of China      194.56          155.65   
8                          Bank of America      231.52          185.22   
9                           JPMorgan Chas